In [1]:
import kuzu

db = kuzu.Database("./final_db")
conn = kuzu.Connection(db)

In [2]:
from graph import find_edges, list_recipes
import pandas as pd

In [3]:
list_recipes()

,n.name,n.display_name
0,surf_turf,Surf & Turf
1,korean_chicken_sandwhiches,Korean Chicken Sandwiches
2,sausage_risotto,Sausage Risotto
3,cherry_tomato_pasta,Cherry Tomato Pasta
4,salmon_bowl,Salmon Bowl
5,taco_bowl,Taco Bowl
6,curry,Curry
7,quinoa,Quinoa
8,vodka_pasta,Vodka Pasta
9,turkey_burger,Turkey Burger


In [67]:
def list_ids(name):
    response = conn.execute(
        """
        MATCH (n)-[r]->(m)
        WHERE n.name = $name
        RETURN m.name;
        """, {"name" : name})
    
    df = response.get_as_df()
    df.columns = ['Ingredient']

    return df['Ingredient'].tolist()

In [16]:
def get_similar_recipes(name):
    response = conn.execute(
        """
        MATCH (n:Recipe)-[r]->(m:Ingredient)
        MATCH (s:Recipe)<-[q]-(p:Ingredient) 
        WHERE n.name = $name
        AND p.name = m.name
        AND s.name <> n.name
        RETURN s.name as name, COUNT(s) as qty;
        """, {"name" : name})
    
    df = response.get_as_df().sort_values(by='qty', ascending=False)
    
    return df
    
    return df

In [65]:
df1 = get_similar_recipes('caesar_salad')
df2 = get_similar_recipes('steak_salad')

print(df1)
print(df2)

                     name  qty
4             vodka_pasta    3
11            tomato_soup    3
2             steak_salad    2
5                  quinoa    2
6         sausage_risotto    2
9     cherry_tomato_pasta    2
0                    blta    1
1   chinese_chicken_salad    1
3           turkey_burger    1
7                   chili    1
8               bolognese    1
10     mediterranean_bowl    1
                          name  qty
12          mediterranean_bowl    4
2                         blta    2
3        chinese_chicken_salad    2
4                 caesar_salad    2
5                turkey_burger    2
7                  vodka_pasta    2
8                       quinoa    2
9          cherry_tomato_pasta    2
10  korean_chicken_sandwhiches    2
0                 steak_potato    1
1                    surf_turf    1
6                    bolognese    1
11                 tomato_soup    1
13                       curry    1


In [8]:
def get_similar_recipes_multi(names):
    print(names)
    response = conn.execute(
        """
        UNWIND $inputRecipes AS inputRecipeName
        MATCH (inputRecipe:Recipe {name: inputRecipeName})-[:Contains]->(inputIngredient:Ingredient)
        WITH inputIngredient, COLLECT(inputRecipe) AS inputRecipes
        MATCH (similarRecipe:Recipe)<-[:UsedIn]-(similarIngredient:Ingredient)
        WHERE similarIngredient.name = inputIngredient.name
        AND NOT similarRecipe IN inputRecipes
        WITH similarRecipe, collect(similarIngredient) AS sharedIngredients
        WHERE size(sharedIngredients) > 0
        RETURN similarRecipe.name, size(sharedIngredients) AS sharedIngredientCount
        ORDER BY sharedIngredientCount DESC;
        """, {"inputRecipes" : names}
    )
    
    df = response.get_as_df()
    
    return df

get_similar_recipes_multi(['steak_salad', 'blta'])

['steak_salad', 'blta']


,similarRecipe.name,sharedIngredientCount
0,mediterranean_bowl,4
1,turkey_burger,3
2,chinese_chicken_salad,2
3,caesar_salad,2
4,vodka_pasta,2
5,cherry_tomato_pasta,2
6,caprese_sandwich,2
7,quinoa,2
8,korean_chicken_sandwhiches,2
9,grilled_cheese,2


In [96]:
df

,inputIngredient,inputRecipes,similarRecipe,similarIngredient


In [93]:
df['inputIngredient'][0]

{'_id': {'offset': 0, 'table': 0},
 '_label': 'Ingredient',
 'name': 'steak',
 'display_name': 'steak',
 'type': ' protein'}

In [71]:
list_ids('chinese_chicken_salad')

['chicken_breast',
 'soy_sauce',
 'seaseme_oil',
 'honey',
 'sriracha',
 'cucumber',
 'apple_cider_vinegar',
 'lettuce',
 'shredded_carrots',
 'ginger',
 'ginger',
 'wonton_strips',
 'tahini']

In [72]:
list_ids('caesar_salad')

['garlic',
 'lemon',
 'olive_oil',
 'lettuce',
 'parm',
 'anchovies',
 'dijon_mustard',
 'croutons',
 'meat']

In [69]:
list_ids('tomato_soup')

['garlic',
 'olive_oil',
 'heavy_cream',
 'parm',
 'onion',
 'tomatos',
 'red_bell_pepper',
 'dried_basil',
 'dried_oregano',
 'chicken_broth',
 'basil']

In [70]:
list_ids('vodka_pasta')

['garlic',
 'lemon',
 'calabrian_chilis',
 'olive_oil',
 'red_onion',
 'pasta',
 'tomato_paste',
 'heavy_cream']

In [76]:
# hard coded similarity

start = 'chinese_chicken_salad'
ing = list_ids(start)
similar = {}
for i in ing:
    # print(i)
    sim = list_ids(i)
    for s in sim:
        if s not in similar:
            similar[s] = 0
        similar[s] += 1

similar

{'korean_chicken_sandwhiches': 5,
 'chinese_chicken_salad': 13,
 'chicken_teriyaki': 1,
 'sausage_risotto': 1,
 'bone_broth_soup': 1,
 'steak_salad': 2,
 'salmon_bowl': 1,
 'quinoa': 2,
 'mediterranean_bowl': 1,
 'short_rib_bowl': 1,
 'turkey_burger': 1,
 'caesar_salad': 1,
 'blta': 2,
 'caprese_sandwich': 1}

In [75]:
ing = list_ids('chicken_thigh')
for i in ing:
    print(i)
    print(list_ids(i))

cherry_tomato_pasta
['garlic', 'angel_hair', 'cherry_tomatos', 'calabrian_chilis', 'olive_oil', 'chicken_thigh']
curry
['rice', 'chicken_thigh', 'curry_paste', 'frozen_veggies', 'red_onion']


In [50]:
find_edges('chicken_thigh')

,Ingredient,Quantity
0,Cherry Tomato Pasta,1
1,Curry,eyeball


In [64]:
find_edges("cherry_tomato_pasta")

[]